In [ ]:
outputFolder=/home/xw2629/noncoding_translation/noncoding-translation-code/U-periodicity

fastqFolder=/media/rna/sequencing_data/jordan/Pep30-Pep13-171229_WIGTC-HISEQ3A_CB6NYANXX/QualityScore

N90Table=/home/xw2629/noncoding_translation/oligo-pool-design/XuebingWu-12K-120-oligo-pool.txt

# P: GFP plus
# M: GFP minus
# R: RNA
# D: DNA

fastqFiles=( "N39P:TTAGGC"
             "N39M:CTTGTA"
             "N90P:AGTTCC"
             "N90M:GAGTGG"
             "N39D:TGACCA"
             "N90D:GCCAAT"
             "N39R:TAGCTT"
             "N90R:ATCACG" )


if [ ! -d "$outputFolder" ]; then
        mkdir $outputFolder
fi

cd $outputFolder



for entry in "${fastqFiles[@]}" ; do
    sample="${entry%%:*}"
    file="${entry##*:}-s_7_1_sequence.txt.gz"
    echo $sample $file
    zcat $fastqFolder/$file | awk 'NR % 4 == 2' | sort | bedtools groupby -g 1 -c 1 -o count > $sample.counts.txt 
done


In [1]:
# clean up Pep13 sequences
# should end with T: read length 40, but only the first 39 are variable
# more than 1 read, and contain no N
for file in N39?.counts.txt
do
    echo $file
    more $file | grep -v N | grep "^.\{39\}T" > $file.filtered
    #more $file.filtered | awk '{sum+=$2}END{print sum}'
done
wc -l N39?.counts.txt*

N39D.counts.txt
N39M.counts.txt
N39P.counts.txt
N39R.counts.txt
  1192652 N39D.counts.txt
   996837 N39D.counts.txt.filtered
  3736799 N39M.counts.txt
  3178572 N39M.counts.txt.filtered
  2766054 N39P.counts.txt
  2353836 N39P.counts.txt.filtered
  3684627 N39R.counts.txt
  3160621 N39R.counts.txt.filtered
 21069998 total


In [2]:
# merge Pep13 count files
intersectMultiTab N39.counts.txt 1 0 N39D.counts.txt.filtered N39R.counts.txt.filtered N39P.counts.txt.filtered N39M.counts.txt.filtered

outputfile = N39.counts.txt
col = 1
fill = 0
N39D.counts.txt.filtered
N39R.counts.txt.filtered
N39P.counts.txt.filtered
N39M.counts.txt.filtered
combining files
N39D.counts.txt.filtered
[Tue Nov 22 18:54:01 2022] 1 columns in all.ids.uniq
[Tue Nov 22 18:54:01 2022] 2 columns in N39D.counts.txt.filtered
[Tue Nov 22 18:54:06 2022] 996837 lines from N39D.counts.txt.filtered
[Tue Nov 22 18:54:46 2022] 8927590 lines from all.ids.uniq
[Tue Nov 22 18:54:46 2022] 996837 lines in common 
N39R.counts.txt.filtered
[Tue Nov 22 18:54:48 2022] 2 columns in all.ids.uniq
[Tue Nov 22 18:54:48 2022] 2 columns in N39R.counts.txt.filtered
[Tue Nov 22 18:55:02 2022] 3160621 lines from N39R.counts.txt.filtered
[Tue Nov 22 18:55:49 2022] 8927590 lines from all.ids.uniq
[Tue Nov 22 18:55:49 2022] 3160621 lines in common 
N39P.counts.txt.filtered
[Tue Nov 22 18:55:51 2022] 3 columns in all.ids.uniq
[Tue Nov 22 18:55:51 2022] 2 columns in N39P.counts.txt.filtered
[Tue Nov 22 18:56:02 2022] 2353836 lines from N3

In [3]:
head more N39.counts.txt 

head: cannot open 'more' for reading: No such file or directory
==> N39.counts.txt <==
AAAAAAAAAAAAAAAAAAAAAAAAAATTAAAAAAAAGATT	0	1	0	0
AAAAAAAAAAAAACTTCCCAGAGCCTGGAGTGGTTTGAAT	0	0	1	0
AAAAAAAAAAAACCTTCCTTAGGCCTGACTTGGTTTTTGT	0	0	0	1
AAAAAAAAAAAATTTTCCCCAAGCCTTTTGTGGTTTTTCT	0	1	0	0
AAAAAAAAAAAATTTTTCAACCAACCTTTTTGGGTGTTCT	0	0	0	1
AAAAAAAAAAAATTTTTCTAAAACATTATTGGGTTGTTCT	0	0	0	1
AAAAAAAAAAAATTTTTTTCAAAACCTTTGTTGTTTTTTT	0	0	0	1
AAAAAAAAAAACGTGTGTTTTTTTTTTTTTTCTTTTTTTT	0	0	0	1
AAAAAAAAAAACTCCTTCAAGACCGTTGTCTGATTTTATT	0	0	0	1
AAAAAAAAAAACTCTTCCCAAAACTTTATGAGGGTATTTT	0	0	0	1


: 1

In [5]:
# get some summary statistics

# number of unique sequences and total reads in GFP-high 
more N39.counts.txt | awk '$4>0' | wc -l
more N39.counts.txt | awk '$4>0' | awk '{s=s+$4}END{print s}'

2353836
21020499


In [6]:
# number of unique sequences and total reads in GFP-low 
more N39.counts.txt | awk '$5>0' | wc -l
more N39.counts.txt | awk '$5>0' | awk '{s=s+$5}END{print s}'

3178572
31388971


In [10]:
# number of unique sequences present in both gfp fractions
more N39.counts.txt | awk '$4>0 && $5>0' | wc -l
more N39.counts.txt | awk '$4>0 || $5>0' | wc -l

more N39.counts.txt | awk '$4>1 && $5>1' | wc -l
more N39.counts.txt | awk '$4>1 || $5>1' | wc -l


66618
5465790
41094
618477


In [6]:
# merge Pep30 count files
N90Table=/home/xw2629/noncoding_translation/old-data/oligo-pool-design/XuebingWu-12K-120-oligo-pool.txt

cat $N90Table | awk '{print substr($2,16,40)"\t"$0}' > tmp

intersectTab -f1 tmp -f2 N90D.counts.txt -merge -o tmp2 -fill 0
intersectTab -f1 tmp2 -f2 N90R.counts.txt -merge -o tmp -fill 0
intersectTab -f1 tmp -f2 N90P.counts.txt -merge -o tmp2 -fill 0
intersectTab -f1 tmp2 -f2 N90M.counts.txt -merge -o tmp -fill 0


[Wed Nov 23 14:49:06 2022] 3 columns in tmp
[Wed Nov 23 14:49:06 2022] 2 columns in N90D.counts.txt
[Wed Nov 23 14:49:12 2022] 1434014 lines from N90D.counts.txt
[Wed Nov 23 14:49:12 2022] 12000 lines from tmp
[Wed Nov 23 14:49:12 2022] 11018 lines in common 
[Wed Nov 23 14:49:12 2022] 4 columns in tmp2
[Wed Nov 23 14:49:12 2022] 2 columns in N90R.counts.txt
[Wed Nov 23 14:49:20 2022] 1637600 lines from N90R.counts.txt
[Wed Nov 23 14:49:20 2022] 12000 lines from tmp2
[Wed Nov 23 14:49:20 2022] 11419 lines in common 
[Wed Nov 23 14:49:20 2022] 5 columns in tmp
[Wed Nov 23 14:49:20 2022] 2 columns in N90P.counts.txt
[Wed Nov 23 14:49:29 2022] 1965593 lines from N90P.counts.txt
[Wed Nov 23 14:49:29 2022] 12000 lines from tmp
[Wed Nov 23 14:49:29 2022] 10819 lines in common 
[Wed Nov 23 14:49:29 2022] 6 columns in tmp2
[Wed Nov 23 14:49:29 2022] 2 columns in N90M.counts.txt
[Wed Nov 23 14:49:35 2022] 1219224 lines from N90M.counts.txt
[Wed Nov 23 14:49:35 2022] 12000 lines from tmp2
[Wed N

In [8]:
cut -f 3- tmp | sed 's/TACTGCGGCCGCTAC//g' | sed 's/TGACTAGCTGACCTG//g' | awk '{print toupper($0)}'> Pep30.counts.txt


In [9]:
head Pep30.counts.txt

CTGCTCAGAAAGTTTGCCACCTCATGGGAATTAATGTGACAGATTTCACCAGATCCATCCTCACTCCTCGTATCAAGGTTGGGCGAGATG	147	501	223	0
CACACACAAAAGACATGCCATTTACATGCGAAACCTGTGGAAAATCATTCAAACGCAGTATGTCACTCAAGGTGCACTCCTTGCAGCATT	1263	1339	870	1077
GATGGAAGGTTAACATTTCATCCAGGAAGTCAGGTAGTGAAACTTCCTTTTATCAACTTTATGAAGACACGTGGGACTTCCTTTCTGAAT	509	822	401	316
GTGGGACAATCCTTCATCCCATCATCGGTGCCTGCAACCTTTGCTCCTTCACCTACACCTGCTGTGGTCAGCAGTGGACTGAATGACCTG	263	414	1175	1
GTCCCCAAAGTCTCTGACCAGGCTTTGGTTTCTGCCCACTCAGAGTGGCAGCGGAAGCTGGAGGCCGCTGAGGCTCTGCTGACTCTGAGA	215	556	259	181
GTTGAAATCGGCTCGAGATGTTGTATCTAGGACTGGTCATTCATTGGCTCTTGGTTGTTTGCATCGTTATGTTGGTGGAATAGGCTCAGG	2589	3821	3803	258
GTACATGGGACAGTAGGAGAACCTTGGTGGATTTGGGTAGAAGATCCTACAAATGATCATATTTATCATTCAGAGTATTTTCTAGCTCTA	228	425	148	1154
GCTGGAGAGCTGCGGTGTGACATCAGACAACTGCCGGGACCTGTGCGGCATTGTGGCCTCCAAGGCCTCGCTGCGGGAGCTGGCCCTGGG	1362	1591	1806	329
AACTTAAACCAGTCAGTAGCCCATGACTGGACATCAGGTTTACAAAGGCTTATTTTGAAGAAAGAAGATGAAATCAGAGCTGCGGACTGC	146	666	1408	2
GCCTCCCGAGGAGCCCAGACAGCTGCAGCCA

In [11]:
rm tmp* all.ids

rm: cannot remove 'tmp*': No such file or directory


: 1